![title.png](https://i.ibb.co/2KmT38V/title.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![objectives.png](https://i.ibb.co/fxbWnNQ/objectives.png)

![image.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Notebook setup and dependency installation

In [1]:
!   pip install -qU \
    openai==1.30 \
    "pinecone-client[grpc]==4.1.0" \
    datasets==2.19 \
    pandas==2.2.2 \
    tqdm

In [2]:
from IPython.display import HTML, display, Markdown
from typing import Dict


def chunk_display_html(chunk: Dict[str, str]) -> str:
    html_template = """
<html>
<head>
<style>
    table {{
        font-family: arial, sans-serif;
        border-collapse: collapse;
        width: 100%;
    }}
    td, th {{
        border: 1px solid #dddddd;
        text-align: left;
        padding: 8px;
    }}
</style>
</head>
<body>
    <table>
        <tr>
            <th>Key</th>
            <th>Value</th>
        </tr>
        <tr>
            <td>Title</td>
            <td>{title}</td>
        </tr>
        <tr>
            <td>doi</td>
            <td>{doi}</td>
        </tr>
        <tr>
            <td>Chunk ID</td>
            <td>{chunk_id}</td>
        </tr>
        <tr>
            <td>chunk</td>
            <td>{chunk}</td>
        </tr>
        <tr>
            <td>id</td>
            <td>{id}</td>
        </tr>
        <tr>
            <td>Summary</td>
            <td>{summary}</td>
        </tr>
        <tr>
            <td>Source</td>
            <td>{source}</td>
        </tr>
        <tr>
            <td>Authors</td>
            <td>{authors}</td>
        </tr>
        <tr>
            <td>Categories</td>
            <td>{categories}</td>
        </tr>
        <tr>
            <td>Comment</td>
            <td>{comment}</td>
        </tr>
        <tr>
            <td>Journal Reference</td>
            <td>{journal_ref}</td>
        </tr>
        <tr>
            <td>Primary Category</td>
            <td>{primary_category}</td>
        </tr>
        <tr>
            <td>Published</td>
            <td>{published}</td>
        </tr>
        <tr>
            <td>Updated</td>
            <td>{updated}</td>
        </tr>
        <tr>
            <td>References</td>
            <td>{references}</td>
        </tr>
    </table>
</body>
</html>
"""

    # Format the HTML with the generated rows
    html_output = html_template.format(
        doi=chunk.get("doi", "N/A"),
        chunk_id=chunk.get("chunk-id", "N/A"),
        chunk=chunk.get("chunk", "N/A"),
        id=chunk.get("id", "N/A"),
        title=chunk.get("title", "N/A"),
        summary=chunk.get("summary", "N/A"),
        source=chunk.get("source", "N/A"),
        authors=chunk.get("authors", "N/A"),
        categories=chunk.get("categories", "N/A"),
        comment=chunk.get("comment", "N/A"),
        journal_ref=chunk.get("journal_ref", "N/A"),
        primary_category=chunk.get("primary_category", "N/A"),
        published=chunk.get("published", "N/A"),
        updated=chunk.get("updated", "N/A"),
        references=chunk.get("references", "N/A"),
    )

    # Display the HTML in an IPython notebook
    display(HTML(html_output))


def display_retrieved_context(context_response):
    # HTML template for the main container and individual tables
    html_template = """
    <html>
    <head>
    <style>
        .container {{
            display: flex;
            flex-wrap: wrap;
        }}
        .table-container {{
            margin: 10px;
            padding: 10px;
            border: 1px solid #dddddd;
        }}
        table {{
            font-family: arial, sans-serif;
            border-collapse: collapse;
            width: 100%;
        }}
        td, th {{
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }}
    </style>
    </head>
    <body>
        <div class="container">
            {tables}
        </div>
    </body>
    </html>
    """

    # Function to generate HTML table for a single dictionary
    def generate_table_for_dict(data):
        rows = "\n".join(
            "<tr><td>{key}</td><td>{value}</td></tr>".format(
                key=key, value=value if value is not None else "N/A"
            )
            for key, value in data.items()
        )
        table_html = """
        <div class="table-container">
            <table>
                <tr>
                    <th>Key</th>
                    <th>Value</th>
                </tr>
                {rows}
            </table>
        </div>
        """.format(
            rows=rows
        )
        return table_html

    # Generate HTML tables for all dictionaries in the list
    tables = "\n".join(
        generate_table_for_dict(data["metadata"]) for data in context_response
    )

    # Format the main HTML with the generated tables
    html_output = html_template.format(tables=tables)

    # Display the HTML in an IPython notebook
    display(HTML(html_output))


def display_markdown(content: str) -> None:
    display(Markdown(content))

![image.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

![step-away-rag.png](https://i.ibb.co/Y288TWR/step-away-rag.png)

## Setup OpenAI

Enter the OpenAI API key and instantiate the OpenAI clinet.

Copy this openai API key into the prompt.
The key is shared here: https://docs.google.com/document/d/1B-UWsusJOjj7n96_3GXUW5iXSn4ErXjxNq--aapP_NI/edit?pli=1 

In [3]:
import getpass
from openai import OpenAI

OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")
openai = OpenAI(api_key=OPENAI_API_KEY)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

## Generate text using OpenAI GPT

In [5]:
def generate(prompt: str, openai_client: OpenAI, model: str = "gpt-3.5-turbo") -> str:

    # API reference: https://platform.openai.com/docs/api-reference/chat/create
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Output is markdown"},
            {"role": "user", "content": prompt},
        ],
        temperature=0.0,
    )

    return response.choices[0].message.content


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [6]:
display_markdown(generate("What is the capital of Germany?", openai))

The capital of Germany is Berlin.

In [7]:
display_markdown(generate("Who is 25th person that landed on the moon?", openai, model="gpt-3.5-turbo"))

The 25th person to land on the moon was Harrison Schmitt. He was part of the Apollo 17 mission in December 1972 and was the only geologist to walk on the moon.

![hallucinations-problem.png](https://i.ibb.co/gMvNZC6/hallucinations-problem.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [8]:
display_markdown(generate("What are key benefits of mistral 7B?", openai, model="gpt-3.5-turbo"))

The Mistral 7B is a powerful and versatile wind turbine that offers several key benefits:

1. **High Efficiency**: The Mistral 7B is designed to maximize energy production with its high-efficiency blades and generator, ensuring optimal performance in various wind conditions.

2. **Reliability**: This wind turbine is built with durable materials and a robust design, making it reliable and able to withstand harsh weather conditions.

3. **Low Maintenance**: The Mistral 7B requires minimal maintenance, reducing operational costs and ensuring consistent performance over its lifespan.

4. **Quiet Operation**: With advanced noise reduction technology, this wind turbine operates quietly, making it suitable for residential and commercial applications.

5. **Scalability**: The Mistral 7B can be easily integrated into existing renewable energy systems or used as a standalone solution, providing flexibility for different energy needs.

Overall, the Mistral 7B offers a combination of efficiency, reliability, low maintenance, quiet operation, and scalability, making it a valuable choice for renewable energy generation.

![knowledge-cutoff.png](https://i.ibb.co/ccccpxZ/knowledge-cutoff.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [9]:
context_example = """
Answer the question based on the following context. If you don't can't find the answer, tell I don't know.

Context:
We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.

Question: What are key benefits of Mistral 7B?
"""
display_markdown(generate(context_example, openai))

The key benefits of Mistral 7B include:

1. Superior performance and efficiency compared to other models.
2. Outperforming the best open 13B model (Llama 2) across all evaluated benchmarks.
3. Outperforming the best released 34B model (Llama 1) in reasoning, mathematics, and code generation.
4. Leveraging grouped-query attention (GQA) for faster inference.
5. Using sliding window attention (SWA) to effectively handle sequences of arbitrary length with reduced inference cost.
6. Providing a fine-tuned model, Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model on both human and automated benchmarks.

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

![why-rag.png](https://i.ibb.co/KF3xr64/why-rag.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![how-to-build-rag.png](https://i.ibb.co/wgXqfFv/how-to-build-rag.png)

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## 1. Build a knowledge base

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![build-knowledge-base.png](https://i.ibb.co/dGnjrCk/build-knowledge-base.png)


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Setup Pinecone 

Enter the Pinecone API key inside the prompt and create a Pinecone client.

In [10]:
PINECONE_REGION = "us-east-1"
PINECONE_CLOUD = "aws"
INDEX_NAME = "pinecone-workshop-1"
VECTOR_DIMENSIONS = 1536
PINECONE_API_KEY = getpass.getpass("Enter your Pinecone API key: ")

In [11]:
from pinecone.grpc import PineconeGRPC
#from pinecone import Pinecone

pinecone = PineconeGRPC(api_key=PINECONE_API_KEY)
# pinecone = Pinecone(api_key=PINECONE_API_KEY)

pinecone.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'pinecone-worshop-1-2kw20wn.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'pinecone-worshop-1',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1536,
              'host': 'pinecone-workshop-1-2kw20wn.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'pinecone-workshop-1',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Create a Pinecone Index

More info o serverless: https://docs.pinecone.io/reference/architecture/serverless-architecture#overview  
Ref API: https://docs.pinecone.io/reference/api/control-plane/create_index 

In [12]:
from pinecone import ServerlessSpec

# Check if the index already exists and delete it
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)

# Create a new index with the specified name, dimension, metric, and spec
# Docs: https://docs.pinecone.io/reference/api/control-plane/create_index
pinecone.create_index(
    name=INDEX_NAME,
    dimension=VECTOR_DIMENSIONS,
    metric="cosine",
    spec=ServerlessSpec(cloud=PINECONE_CLOUD, region=PINECONE_REGION),
)

# Create a new Index reference object with the specified name
index = pinecone.Index(INDEX_NAME)
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Dataset

We are going to use a sample of 1000 AI papers that can be found here: https://huggingface.co/datasets/smartcat/ai-arxiv2-chunks-embedded 

The data set is already chunked and encoded using `text-embeddings-3-small` so we can just load and upsert it to the Pinecone.

If you want to play with chunking strategies and embeddings, you can find the full data set here: https://huggingface.co/datasets/jamescalam/ai-arxiv2

Dataset API reference: https://huggingface.co/docs/datasets/en/index  
Slicing and indexing: https://huggingface.co/docs/datasets/en/access 

In [13]:
import datasets

dataset = datasets.load_dataset("smartcat/ai-arxiv2-chunks-embedded", split="train")
dataset

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references', 'embeddings', 'metadata'],
    num_rows: 79782
})

In [14]:
chunk_display_html(dataset[0])

Key,Value
Title,Foundations of Vector Retrieval
doi,2401.09350
Chunk ID,0
chunk,"4 2 0 2 n a J 7 1 ] S D . s c [ 1 v 0 5 3 9 0 . 1 0 4 2 : v i X r a Sebastian Bruch # Foundations of Vector Retrieval # Preface We are witness to a few years of remarkable developments in Artificial Intelligence with the use of advanced machine learning algorithms, and in particular, deep learning. Gargantuan, complex neural networks that can learn through self-supervisionâand quickly so with the aid of special- ized hardwareâtransformed the research landscape so dramatically that, incremental overnight it seems, many fields experienced not the usual, progress, but rather a leap forward. Machine translation, natural language understanding, information retrieval, recommender systems, and computer vision are but a few examples of research areas that have had to grapple with the shock. Countless other disciplines beyond computer science such as robotics, biology, and chemistry too have benefited from deep learning."
id,2401.09350#0
Summary,"Vectors are universal mathematical objects that can represent text, images, speech, or a mix of these data modalities. That happens regardless of whether data is represented by hand-crafted features or learnt embeddings. Collect a large enough quantity of such vectors and the question of retrieval becomes urgently relevant: Finding vectors that are more similar to a query vector. This monograph is concerned with the question above and covers fundamental concepts along with advanced data structures and algorithms for vector retrieval. In doing so, it recaps this fascinating topic and lowers barriers of entry into this rich area of research."
Source,http://arxiv.org/pdf/2401.09350
Authors,Sebastian Bruch
Categories,"cs.DS, cs.IR"
Comment,None


In [15]:
print(len(dataset[0]["embeddings"]))
print(dataset[0]["embeddings"][:10])

1536
[-0.020014504, -0.013545036, 0.04353385, -0.0029185074, 0.03552278, -0.034943033, 0.013927143, 0.06566971, -0.06888468, 0.026971487]


In [16]:
list(dataset[0]["metadata"].keys())

['authors',
 'chunk_id',
 'doc_id',
 'primary_category',
 'published',
 'source',
 'summary',
 'text',
 'title',
 'year']

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### What has been done with data set?

![chunking-dataset.png](https://i.ibb.co/9wV70Q7/chunking-dataset.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

#### Chunking Strategies
1. Character split (with overlapping)
2. Recursive character split
3. Document specific splitting
4. Semantic Chunking
5. Agentic?
6. More?

Introduction to chunking: https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Data upsert to Pinecone

Let insert data to the Pinecone in batches. 

From our data set we need 3 columns:
1. `id` - the ID of the chunk we want to insert
2. `embeddings` - contains a vector embedding of the chunk. It uses `text-embeddings-3-small`
3. `metadata` - a dictinary with additional data about the chunk. 

The code for upserting:
```python
index.upsert(vectors=[ 
    (id1, vector1, metadata1),
    (id2, vector2, metadata2),
    ....
 ])
```
We can also upsert data directly from the `pandas.Dataframe` using `index.upsert_from_dataframe`.  
It requires 3 columns: 
1. `values` - a column that contain a embeddings for each chunk/record
2. `id` - an ID of a record
3. `metadata` - a key-value pairs for record metadata

Upsert from dataframe: https://docs.pinecone.io/guides/data/use-public-pinecone-datasets#upsert-a-dataset-as-a-dataframe 
Avoid quotas and limits: https://docs.pinecone.io/reference/quotas-and-limits  
For scale-up and optimizations make sure to read: : https://docs.pinecone.io/guides/operations/performance-tuning#increasing-throughput  
Metadata filtering: Metadata filtering: https://docs.pinecone.io/guides/data/filter-with-metadata 

In [18]:
from pinecone.grpc import GRPCIndex as Index

def upsert_batch(ds: datasets.Dataset, index: Index, batch_size: int = 200) -> None:
    df = ds.to_pandas()
    df = df[["id", "embeddings", "metadata"]]
    df.rename(columns={"embeddings": "values"}, inplace=True)
    index.upsert_from_dataframe(df, batch_size=batch_size, show_progress=True)

In [19]:
upsert_batch(dataset.select(range(500)), index, batch_size=100)

sending upsert requests:   0%|          | 0/500 [00:00<?, ?it/s]

collecting async responses:   0%|          | 0/5 [00:00<?, ?it/s]

In [20]:
print(index.describe_index_stats())

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 500}},
 'total_vector_count': 500}


In [21]:
upsert_batch(dataset, index, batch_size=150)

sending upsert requests:   0%|          | 0/79782 [00:00<?, ?it/s]

collecting async responses:   0%|          | 0/532 [00:00<?, ?it/s]

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Revisit Agenda


![done-next.png](https://i.ibb.co/DKX2cKz/done-next.png)


![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## 2. Retrieve against the query



![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![semantic-search.png](https://i.ibb.co/2dWGRHn/semantic-search.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

Once we inserted everythong to Pinecone, let's query it. The input is query text and the output is the top similar chunks.
Steps: 
1. Encode the input text to generate embeddings 
2. Call Pinecone's query function to retrieve top K similar results

In [22]:
from typing import List


def encode(
    text: str, openai_client: OpenAI, model: str = "text-embedding-3-small"
) -> List[float]:
    # Use the OpenAI API to encode the text
    # Docs: https://platform.openai.com/docs/api-reference/embeddings
    response = openai_client.embeddings.create(
        model=model, input=text
    )

    return response.data[0].embedding

In [23]:
res = encode("What are key benefits of mistral 7B?", openai)
print(res[:10])
print(len(res))

[-0.02581052854657173, 0.03242715075612068, 0.019921164959669113, -0.021119002252817154, -0.036020658910274506, 0.023999514058232307, -0.05059434100985527, 0.05886511504650116, 0.024940671399235725, -0.006969555746763945]
1536


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Query Pinecone

In [24]:
from pinecone import QueryResponse


def retrieve(
    query: str, index: Index, openai_client: OpenAI, top_k: int = 10
) -> QueryResponse:
    # Encode the query using the OpenAI API
    query_embedding = encode(query, openai_client)

    # Use the Pinecone index to query the encoded vector
    # Docs: https://docs.pinecone.io/reference/api/data-plane/query
    ret = index.query(
        vector=query_embedding, top_k=top_k, include_metadata=True, include_values=False
    )
    return ret

In [25]:
res = retrieve("What is Mistral 7B?", index, openai)
print(res.matches[0].metadata['text'])

Abstract
We introduce Mistral 7B, a 7âbillion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms the best open 13B model (Llama 2) across all evaluated benchmarks, and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding window attention (SWA) to effectively handle sequences of arbitrary length with a reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B â Instruct, that surpasses Llama 2 13B â chat model both on human and automated benchmarks. Our models are released under the Apache 2.0 license. Code: https://github.com/mistralai/mistral-src Webpage: https://mistral.ai/news/announcing-mistral-7b/
# Introduction


![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## 3. Augment and generate

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

![workflow-rag-simple.png](https://i.ibb.co/p0gwY23/workflow-rag-simple.png)

![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### Generate a final response

Combine all pieces together:
1. Perform a semantic search on input query
2. Build a context (prompt) for a LLM
3. Call LLM to generate a final response
4. Return a final response and retrieved context

The relevant context can be found in metadata, you can use:
1. `title` - a Paper title
2. `published` - a publish date
3. `primart_category` 
4. `summary` - a paper summary
5. `text` - a chunk text - this is the most useful info to build a context for LLM

In [26]:
from typing import Tuple

def from_metadata(metadata: Dict) -> str:
    return f"""
***
    Title: {metadata['title']}
    Authors: {metadata['authors'][:3]}
    Published: {metadata['published']}
    Paper summary: {metadata['summary']}
    Text: {metadata['text']}
    Source: {metadata['source']}

"""


def augment(query: str, query_results: QueryResponse) -> str:
    context = "\n".join(
        [from_metadata(result.metadata) for result in query_results.matches]
    )
    return f"""
Answer the question based on the following context. If you don't can't find the answer, tell I don't know.
The answers should be clear, easy to understand, complete and comprehensive.

Context:
{context}

Question: {query}
"""


def rag(query: str, index: Index, openai_client: OpenAI, top_k: int = 5) -> Tuple[str, QueryResponse]:
    # 1. [RETRIEVE]: Reuse `semantic_search` function to get the top_k results
    # 2. [AUGMENT]: Use `get_prompt` function to generate the prompt (context + question)
    # 3. [GENERATE]: Use `llm_completion` function to generate the response
    # 4. Return the response and query_results
    query_results = retrieve(query, index, openai_client, top_k=top_k)
    prompt = augment(query, query_results)
    response = generate(prompt, openai_client)
    return response, query_results

In [27]:
answer, context = rag("What are key benefits of Mistral 7B?", index, openai)
display_markdown(answer)

The key benefits of Mistral 7B are:

1. **Superior Performance**: Mistral 7B is engineered for superior performance, outperforming other models like Llama 2 13B and Llama 1 34B across various benchmarks, including reasoning, mathematics, and code generation.

2. **Efficiency**: Mistral 7B is designed to be efficient, balancing high performance with reduced computational costs and inference latency. This efficiency makes it suitable for deployment in practical, real-world scenarios.

3. **Innovative Attention Mechanisms**: Mistral 7B leverages grouped-query attention (GQA) for faster inference and sliding window attention (SWA) to handle sequences of arbitrary length effectively. These attention mechanisms contribute to the model's enhanced performance and efficiency.

4. **Ease of Deployment and Fine-Tuning**: Mistral 7B is released under the Apache 2.0 license, accompanied by a reference implementation for easy deployment on cloud platforms. It is also crafted for ease of fine-tuning across a wide range of tasks, making it adaptable for various applications.

5. **Superior Performance in Specific Tasks**: Mistral 7B excels in tasks such as reasoning, mathematics, and code generation, showcasing its versatility and effectiveness in specialized domains.

6. **Compatibility and Integration**: Mistral 7B can be integrated with tools like the vLLM inference server, SkyPilot, and Hugging Face for streamlined deployment and usage, enhancing its compatibility with existing frameworks and platforms.

In [28]:
display_retrieved_context(context.matches[:2])

Key,Value
text,"Mistral 7B is released under the Apache 2.0 license. This release is accompanied by a reference implementation1 facilitating easy deployment either locally or on cloud platforms such as AWS, GCP, or Azure using the vLLM [17] inference server and SkyPilot 2. Integration with Hugging Face 3 is also streamlined for easier integration. Moreover, Mistral 7B is crafted for ease of fine-tuning across a myriad of tasks. As a demonstration of its adaptability and superior performance, we present a chat model fine-tuned from Mistral 7B that significantly outperforms the Llama 2 13B â Chat model. Mistral 7B takes a significant step in balancing the goals of getting high performance while keeping large language models efficient. Through our work, our aim is to help the community create more affordable, efficient, and high-performing language models that can be used in a wide range of real-world applications. # 2 Architectural details The cat sat on the The cat sat on the window size â_ââ> The cat sat on the Vanilla Attention Sliding Window Attention Effective Context Length"
chunk_id,3.0
year,2023.0
primary_category,cs.CL
authors,"Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed"
source,http://arxiv.org/pdf/2310.06825
summary,"We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered for superior performance and efficiency. Mistral 7B outperforms Llama 2 13B across all evaluated benchmarks, and Llama 1 34B in reasoning, mathematics, and code generation. Our model leverages grouped-query attention (GQA) for faster inference, coupled with sliding window attention (SWA) to effectively handle sequences of arbitrary length with a reduced inference cost. We also provide a model fine-tuned to follow instructions, Mistral 7B -- Instruct, that surpasses the Llama 2 13B -- Chat model both on human and automated benchmarks. Our models are released under the Apache 2.0 license."
published,20231010.0
title,Mistral 7B
doc_id,2310.06825


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

In [30]:
answer, context = rag("What is skeleton of thoughts?", index, openai)
display_markdown(answer)

The "Skeleton-of-Thought" (SoT) is a methodology proposed in the research paper titled "Skeleton-of-Thought: Large Language Models Can Do Parallel Decoding" by Xue. The main objective of SoT is to reduce the end-to-end generation latency of large language models (LLMs) by changing the traditional sequential decoding approach to a parallel decoding approach. This approach is inspired by how humans think and write answers, where the idea is to guide the LLM to first generate a concise skeleton of the answer and then complete the contents of each skeleton point in parallel.

### Methodology of Skeleton-of-Thought (SoT)
1. **Skeleton Stage**: In this stage, SoT assembles a skeleton request using a skeleton prompt template with the question as a parameter. The LLM is guided to output a brief skeleton of the answer, and then B points are extracted from this skeleton response.
   
2. **Point-Expanding Stage**: Based on the skeleton, the LLM expands on each point in parallel. For each point, a point-expanding request is made to the LLM using a point-expanding prompt template. After completing all points, the point-expanding responses are concatenated to form the final answer.

### Benefits of Skeleton-of-Thought (SoT)
- SoT provides significant speed-ups across 12 LLMs.
- It has the potential to enhance the answer quality on various question categories.
- SoT is an initial step towards data-centric optimization for inference efficiency, aiming to make LLMs think more like humans for improved answer quality.

In summary, the Skeleton-of-Thought methodology introduces a structured approach to answer generation by first creating a skeleton and then expanding on it in parallel, which can lead to faster response times and potentially higher-quality answers across different question categories.

In [31]:
answer, context = rag("How to apply LLMs to recommender engines?", index, openai)
display_markdown(answer)

To apply Large Language Models (LLMs) to recommender engines, several approaches can be considered based on the information provided in the context:

1. **LLMs as Recommendation Models**:
   - LLMs can be directly adapted to serve as recommendation models. This approach involves training LLMs to complete the recommendation task without parameter tuning. Various prompt engineering methods can be employed to improve recommendation performance and reduce potential biases.
   
2. **LLM-enhanced Recommendation Models**:
   - In this approach, the universal knowledge encoded in LLMs is leveraged to enhance traditional recommender systems. This can be achieved through:
     - Inferring users' potential intentions from historical interaction data using LLMs and employing these inferred intentions to improve item retrieval in recommendation systems.
     - Using LLMs as feature encoders to encode side information of items and users, such as item descriptions and user reviews, to provide more informative representations for traditional recommender systems.
     - Adopting a distillation-like method to transfer LLM capacities, such as semantic encoding, to improve traditional recommenders by aligning hidden states of LLMs and traditional recommendation models through joint training.

3. **LLMs as Recommendation Simulators**:
   - LLMs can also be utilized as recommendation simulators to simulate potential user instructions in various scenarios like product search and personalized recommendations. This can help in improving the diversity of instructions and enhancing the recommendation process.

4. **Addressing Challenges**:
   - Despite the potential benefits of using LLMs in recommender systems, there are challenges that need to be addressed, such as:
     - Performance issues in zero/few-shot settings compared to traditional ID-based recommenders.
     - Semantic gap between LLMs and recommender systems, leading to high tuning costs and reduced understanding of personalized user behaviors.
     - Inference latency and memory overhead challenges, especially in low-resourced environments.
     - Limited capacities of existing LLMs in long context modeling, requiring improved approaches for processing large amounts of user-item interaction data.

By considering these approaches and challenges, developers can effectively apply LLMs to recommender engines, enhancing recommendation performance and user experience.

In [32]:
display_retrieved_context(context.matches[:2])

Key,Value
text,"LLM-enhanced Recommendation Models. In addition to instructing LLMs to directly provide recommendations, re- searchers also propose leveraging the universal knowledge encoded in LLMs to improve traditional recommender sys- tems. Existing approaches in this line can be divided into three main categories. The first category employs LLMs to infer usersâ potential intention from their historical interac- tion data. Furthermore, traditional recommendation/search models employ the inferred intentions to improve the re- trieval of relevant items [812, 813]. Additionally, several studies explore the use of LLMs as feature encoders. They employ LLMs to encode the side information of items and 75 users (e.g., itemâs descriptions and userâs reviews), thus de- riving more informative representations of users and items. These representations are then fed into traditional recom- mender systems as augmented input [814, 815]. As an- other alternative approach, several studies [816, 817] adopt a distillation-like way to transfer LLMâs capacities (e.g., semantic encoding) to improve traditional recommenders (i.e., small models). Specially, they align the hidden states of LLMs and traditional recommendation models via joint training. After training, since only the enhanced small model will be deployed online, it can avoid the huge over- head of LLMs in online service."
chunk_id,481.0
year,2023.0
primary_category,cs.CL
authors,"Wayne Xin Zhao, Kun Zhou, Junyi Li, Tianyi Tang, Xiaolei Wang, Yupeng Hou, Yingqian Min, Beichen Zhang, Junjie Zhang, Zican Dong, Yifan Du, Chen Yang, Yushuo Chen, Zhipeng Chen, Jinhao Jiang, Ruiyang Ren, Yifan Li, Xinyu Tang, Zikang Liu, Peiyu Liu, Jian-Yun Nie, Ji-Rong Wen"
source,http://arxiv.org/pdf/2303.18223
summary,"Language is essentially a complex, intricate system of human expressions governed by grammatical rules. It poses a significant challenge to develop capable AI algorithms for comprehending and grasping a language. As a major approach, language modeling has been widely studied for language understanding and generation in the past two decades, evolving from statistical language models to neural language models. Recently, pre-trained language models (PLMs) have been proposed by pre-training Transformer models over large-scale corpora, showing strong capabilities in solving various NLP tasks. Since researchers have found that model scaling can lead to performance improvement, they further study the scaling effect by increasing the model size to an even larger size. Interestingly, when the parameter scale exceeds a certain level, these enlarged language models not only achieve a significant performance improvement but also show some special abilities that are not present in small-scale language models. To discriminate the difference in parameter scale, the research community has coined the term large language models (LLM) for the PLMs of significant size. Recently, the research on LLMs has been largely advanced by both academia and industry, and a remarkable progress is the launch of ChatGPT, which has attracted widespread attention from society. The technical evolution of LLMs has been making an important impact on the entire AI community, which would revolutionize the way how we develop and use AI algorithms. In this survey, we review the recent advances of LLMs by introducing the background, key findings, and mainstream techniques. In particular, we focus on four major aspects of LLMs, namely pre-training, adaptation tuning, utilization, and capacity evaluation. Besides, we also summarize the available resources for developing LLMs and discuss the remaining issues for future directions."
published,20230331.0
title,A Survey of Large Language Models
doc_id,2303.18223


![section-breakpoint.png](https://i.ibb.co/344JqH3/section-breakpoint.png)

### What about comparisons?

In [33]:
answer, context = rag(
    "Write side by side summaries between mistral, kosmos, palm. If there is key difference between them, what is it",
    index,
    openai,
    top_k=10,
)
display_markdown(answer)

I don't know the answer to this question as the context provided does not contain information about the models Mistral, Kosmos, and Palm for a side-by-side comparison.

In [34]:
display_retrieved_context(context.matches[:3])

Key,Value
text,Task Zero-shot One-shot Few-shot (k = 4) LLM KOSMOS-1 LLM KOSMOS-1 LLM KOSMOS-1 StoryCloze HellaSwag 72.9 50.4 72.1 50.0 72.9 50.2 72.2 50.0 73.1 50.4 72.3 50.3 Winograd Winogrande 71.6 56.7 69.8 54.8 71.2 56.7 68.4 54.5 70.9 57.0 69.8 55.7 PIQA 73.2 72.9 73.0 72.5 72.6 72.3 BoolQ CB COPA 56.4 39.3 68.0 56.4 44.6 63.0 55.1 41.1 69.0 57.2 48.2 64.0 58.7 42.9 69.0 59.2 53.6 64.0 Average 61.1 60.5 61.2 60.9 61.8 62.2 Table 13: Performance comparisons of language tasks between KOSMOS-1 and LLM. We use the same textual data and training setup to reimplement a language model. Both models do not use instruction tuning for fair comparisons. # 4.9 Cross-modal Transfer
chunk_id,53.0
year,2023.0
primary_category,cs.CL
authors,"Shaohan Huang, Li Dong, Wenhui Wang, Yaru Hao, Saksham Singhal, Shuming Ma, Tengchao Lv, Lei Cui, Owais Khan Mohammed, Barun Patra, Qiang Liu, Kriti Aggarwal, Zewen Chi, Johan Bjorck, Vishrav Chaudhary, Subhojit Som, Xia Song, Furu Wei"
source,http://arxiv.org/pdf/2302.14045
summary,"A big convergence of language, multimodal perception, action, and world modeling is a key step toward artificial general intelligence. In this work, we introduce Kosmos-1, a Multimodal Large Language Model (MLLM) that can perceive general modalities, learn in context (i.e., few-shot), and follow instructions (i.e., zero-shot). Specifically, we train Kosmos-1 from scratch on web-scale multimodal corpora, including arbitrarily interleaved text and images, image-caption pairs, and text data. We evaluate various settings, including zero-shot, few-shot, and multimodal chain-of-thought prompting, on a wide range of tasks without any gradient updates or finetuning. Experimental results show that Kosmos-1 achieves impressive performance on (i) language understanding, generation, and even OCR-free NLP (directly fed with document images), (ii) perception-language tasks, including multimodal dialogue, image captioning, visual question answering, and (iii) vision tasks, such as image recognition with descriptions (specifying classification via text instructions). We also show that MLLMs can benefit from cross-modal transfer, i.e., transfer knowledge from language to multimodal, and from multimodal to language. In addition, we introduce a dataset of Raven IQ test, which diagnoses the nonverbal reasoning capability of MLLMs."
published,20230227.0
title,Language Is Not All You Need: Aligning Perception with Language Models
doc_id,2302.14045


![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

## Hands-On Exercises

### Exercise #1 - Query Comprehension

Your task is to figure out how to execute the following query:  

`Write side by side summaries between mistral, kosmos, palm. If there is key difference between them, what is it`

Hints:
1. JSON response from GPT: https://platform.openai.com/docs/api-reference/chat/create 
2. Use LLM to extract information from the query
3. Apply the right techniques of RAG to construct the best response


In [37]:
# TODO: Implement a query comprehension tool



### Exercise #2 - Paper listing and filtering

Adapt RAG model to work on paper listing, ie it should be able to answer this type of question:  
`List me papers names and sources that are published in 2023 about e-commerce search techniques`

Hints:
1. Use metadata filtering: https://docs.pinecone.io/guides/data/filter-with-metadata (`metadata_filter={"published": {"gt": 20230101}}`)
2. Use LLM (prompt engineering) to extract the useful data from the query (search term, date, task, etc)
3. JSON reponse from ChatGPT - https://platform.openai.com/docs/api-reference/chat/create   

In [36]:
# TODO: Implement paper listing and metadata filtering

## Appendix

### LangChain

An example how simple RAG application could be done with frameworks.

Langchain: https://www.langchain.com/langchain 

```python
!pip install -qU \
    langchain-pinecone \
    langchain-openai \
    langchain
```

```python
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY,
)

vectorstore = PineconeVectorStore(
    index=index, embedding=embeddings, text_key="text", pinecone_api_key=PINECONE_API_KEY, index_name=INDEX_NAME,)

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo", temperature=0.0)
langchain_rag = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever()
)
```

```python
langchain_rag.invoke("What are all benefits of using Mistral 7B?")
```

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)

### Fine-Tune vs RAG vs Prompt Engineering

![rag-vs-prompt-vs-finetune.png](https://i.ibb.co/C8rvnTY/Screenshot-2024-05-30-at-21-17-45.png)

![visual-breakpoint.png](https://i.ibb.co/rHVSp3w/visual-breakpoint.png)